<center><img src="https://github.com/FASSt-simulation/fasst_simulation_tools/raw/main/images/site-logo.png" width=150 height=150 alt="NGEE-Arctic Logo"/></center>

# Tutorial: *Evaluating ELM fluxes using remote sensing imagery from Google Earth Engine*
This tutorial explores the radiation components of ELM simulations at one of the four [NGEE-Arctic](https://ngee-arctic.ornl.gov/) study sites.


#### ----------------------------------------------------------------------------------------------------------------------

<br>

#### To run this script and generate the resulting ELM plots we need specific Python libraries.  This step loads those required libraries

In [ ]:
# setup modules
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import xarray
from netCDF4 import Dataset 
import os
import cftime
from datetime import datetime as dt
import ipywidgets as widgets
import sys,os,os.path,glob

# spatial modules
import ee
import geemap.colormaps as cm
import geemap.foliumap as geemap
    
import warnings
warnings.filterwarnings('ignore') #don't display warnings

<br>

#### This step creates a temporary folder location to store figures that are created in this notebook.  You can later download these figures to your host computer

In [ ]:
## setup output. defined dir or system temp dir
figdir=os.path.expanduser(os.path.join('~','work/figures'))

if not os.path.exists(figdir):
    os.makedirs(figdir)

print(" ")
print("****** Temporary Output Folder ******")
print(figdir)

savefigs = True # True/False

<br>

#### To use the GEE layers you need to have a GEE account and authenticate your API before proceeding

In [ ]:
##### initialize ee API  - you need to authenticate with GEE and initialize

## Trigger the authentication flow. You only need to do this once while running notebook
ee.Authenticate()

## After inserting the API key initialize GEE
ee.Initialize()

<br>

#### This step identifies all of the previous ELM cases that are stored in your output location then creates a list to select the output by site

In [ ]:
# Create case run output directories
output_rootdir=os.path.expanduser('~')+'/output/cime_run_dirs/'
cases=np.asarray(glob.glob("%s*20TR*" % output_rootdir))
cases=[x.split('/')[-1] for x in cases]
cases_dropdown = widgets.Dropdown(options=cases,
                                description='Choose Case Name:',
                                style={'description_width':'auto'},
                                layout={'width':'max-content'},
                                disabled=False)

<br>

#### Here we create the dropdown menu of ELM output options availible in your output volume

In [ ]:
# Show a dropdown menu to select specific case output
# Any cases that have been run in our elmoutput directory will be displayed
# Case names will contain the site codes:
# AK-BEO; AK-CLG; AK-K64G AK-TLG

display(cases_dropdown)

<br>

#### After selecting which ELM output folder we want to use as our source for simulation output, we can then find all .nc files in the output location

In [ ]:
# Get the output nc file options
output_casedir=output_rootdir+cases_dropdown.value+'/run/'
filenames=np.asarray(glob.glob("%s*.elm.h?.*.nc" % output_casedir))
filenames=sorted([x.split('/')[-1] for x in filenames])
if(os.path.exists(output_casedir+'ELM_output.nc')): filenames.insert(0,'ELM_output.nc')

ncfiles_dropdown = widgets.Dropdown(options=filenames,
                                description='Choose Output File:', 
                                style={'description_width':'auto'},
                                layout={'width':'max-content'},
                                disabled=False)

<br>

#### This step displays the .nc file dropdown menu. In most cases you will select "ELM_output.nc" which is the file containing all simulation output years (e.g. 1850-2014)

In [ ]:
# Display the output nc file options - in most cases you will select ELM_output.nc

display(ncfiles_dropdown)

<br>

#### This step uses the xarray package to load the ELM output so that we can analyze the results and generate plots

In [ ]:
# Load model output data into xarray format. 
# squeeze removes an empty grid cell dimension assuming this is a single point run
output_file=output_casedir+ncfiles_dropdown.value
elm_output=xarray.open_dataset(output_file).squeeze()
temptime = elm_output.indexes['time'].to_datetimeindex()
elm_output['time'] = temptime

<br>

#### We can quickly display a subset of the nc file metadata here using the print() command

#### Note that on the left is the name of the data dimension or variable and on the right shows the data type and other metadata

In [ ]:
# show the contents of elm_output

print(elm_output)

<br>

#### Lets get the specific lat/long of the model simulation to use with extracting remote sensing data

In [ ]:
print(elm_output['lon'],elm_output['lat'])
lat=elm_output['lat'].values.item()
lon=elm_output['lon'].values.item()
print(lat,lon)

<br>

#### You will see the lat/long location is inaccurate given the resolution of the global ELM inputs. Instead lets set a more accurate location of the study site based on the list below.  Copy and paste the updated lat and long values below based on which site you are analyzing

AK-BEOG (Utqiagvik): 71.281054, -156.615273 <br>
AK-CLG (Council): 64.854151, -163.726455 <br>
AK-K64G (Kougarok): 65.163724, -164.817472 <br>
AK-TLG (Teller): 64.732523, -165.952788 <br>

In [ ]:
# set the desired lat/long here (copy and paste the lat/longs)
lat=64.732523
lon=-165.952788

<br>

#### Lets get some imagery from GEE. In this case we will make an NDVI map with the site location shown as a black dot

In [ ]:
##### Get Landsat 8 NDVI max data
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_NDVI').filterDate('2019-07-01', '2019-09-05')
l8_ndvi = l8.select('NDVI');

# Composite and drop values == 1
l8_maxval_ndvi_composite = l8_ndvi.max()
l8mask = l8_maxval_ndvi_composite.lt(1)
l8_maxval_ndvi_composite = l8_maxval_ndvi_composite.updateMask(l8mask)

In [ ]:
# Create an NDVI map w/ the site location

# site lat/long
point = ee.Geometry.Point(lon, lat);

ndvi_palette = cm.palettes.viridis
ndviViz = {'min': 0.0, 'max': 0.9, 'palette': ndvi_palette, 'opacity': 1}
l8_ndvi_map = geemap.Map(center=[lat,lon], zoom=10);
l8_ndvi_map.addLayer(l8_maxval_ndvi_composite, ndviViz, 'Max NDVI')
l8_ndvi_map.addLayer(point,{'color': 'black'},'Geometry [black]: point');
l8_ndvi_map.add_colorbar(vis_params=ndviViz, label='Landsat 8 NDVI (-)')
l8_ndvi_map

<br>

#### Before we move on, lets setup a function to extract a timeseries dataset from a specific lat/long location

In [ ]:
def extract_time_series(lat, lon, start, end, product_name, band_name, sf):

    # Set up point geometry
    point = ee.Geometry.Point(lon, lat)

    # Obtain image collection for all images within query dates
    coll = ee.ImageCollection(product_name)\
        .filterDate(start, end)

    # Get list of images which correspond with the above
    images = [item.get('id') for item in coll.getInfo().get('features')]

    store = []
    date_store = []

    # Loop over all images and extract pixel value
    for image in images:

        im = ee.Image(image)
        projection = im.projection().getInfo()['crs']

        # Obtain date from timestamp in metadata
        date = dt.fromtimestamp(im.get("system:time_start").getInfo() / 1000.)
        date_store.append(np.datetime64(date))

        # Extract pixel value
        data = im.select(band_name)\
        .reduceRegion(ee.Reducer.first(),
                      point,
                      1,
                      crs=projection)\
        .get(band_name)
        
        store.append(data.getInfo())

    # Scale the returned data based on scale factor
    store = [x * sf if isinstance(x, int) else np.nan for x in store]

    # Convert output into pandas data frame
    df = pd.DataFrame(index=date_store, data=store, columns=[band_name])

    return df

<br>

#### Lets extract a timeseries of LAI data for the study site

In [ ]:
# Run the extract function
latitude = lat
longitude = lon
start_date = '2010-01-01'
end_date = '2014-12-31'
product = 'MODIS/061/MOD15A2H' 
#LAI scale factor: 0.1
#FPAR scale factor: 0.01
#https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD15A2H

# Extract data and obtain pd.DataFrame
band = 'Lai_500m'
scale_factor = 0.1
modis_lai = extract_time_series(latitude,longitude,start_date,end_date,product,band,scale_factor)

band = 'Fpar_500m'
scale_factor = 0.01
modis_fpar = extract_time_series(latitude,longitude,start_date,end_date,product,band,scale_factor)


# !!! should also extract SDs and plot as a shaded region
#FparStdDev_500m
#Standard deviation of FPAR

#LaiStdDev_500m
#Standard deviation of LAI

<br>

#### Lets look at the data that is returned from the extract function

In [ ]:
modis_lai.head()

In [ ]:
modis_fpar.head()

<br>

#### Lets plot the MODIS LAI and FPAR datasets for the selected years

In [ ]:
#### Plot the LAI and FPAR data

fig = plt.figure(figsize=(19, 6))
ax = fig.add_subplot(1, 2, 1)
modis_lai.where(modis_lai['Lai_500m']<12).plot(ax=ax, kind='line',linewidth=2,color='black')
ax.get_legend().remove()
plt.xticks(fontsize=20, rotation=0, horizontalalignment='center')
plt.yticks(fontsize=20, rotation=0)
ax.set_title('MODIS LAI', size=24)
ax.set_xlabel('Date', size=24)
ax.set_ylabel('LAI (m$^{2}$ m$^{-2}$)', size=24)

ax = fig.add_subplot(1, 2, 2)
modis_fpar.where(modis_fpar['Fpar_500m']<1).plot(ax=ax, kind='line',linewidth=2,color='black')
ax.get_legend().remove()
plt.xticks(fontsize=20, rotation=0, horizontalalignment='center')
plt.yticks(fontsize=20, rotation=0)
ax.set_title('MODIS FPAR', size=24)
ax.set_xlabel('Date', size=24)
ax.set_ylabel('MODIS FPAR (-)', size=24)
if savefigs:
    plt.savefig(os.path.join(figdir,'MODIS_LAI_FPAR.pdf'))
    plt.savefig(os.path.join(figdir,'MODIS_LAI_FPAR.png'))
plt.show()

In [ ]:
timerange=slice('2010-01-01','2014-12-31')

fig = plt.figure(figsize=(20, 8))
ax = fig.add_subplot(1, 1, 1)
modis_lai['Lai_500m'].where(modis_lai['Lai_500m']<12).plot(ax=ax,linestyle='-',linewidth=2,color='black',label='MODIS LAI')
elm_output['TLAI'].sel(time=timerange).plot(ax=ax, linestyle='-',linewidth=4,color='green',label='ELM TLAI')
ax.legend(fontsize=17)
plt.xticks(fontsize=20, rotation=0, horizontalalignment='center')
plt.yticks(fontsize=20, rotation=0)
ax.set_title('LAI: 2010-2014', size=18)
ax.set_xlabel('Date', size=24)
ax.set_ylabel('LAI (m$^{2}$ m$^{-2}$)', size=24)
if savefigs:
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_LAI_2.pdf'))
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_LAI_2.png'))
plt.show()

<br>

#### Lets extract a timeseries of Gross Primary Productivity (GPP) data for the study site

In [ ]:
# Run the extract function
latitude = lat
longitude = lon
start_date = '2010-01-01'
end_date = '2014-12-31'
product = 'MODIS/006/MOD17A2H' 

# Extract data and obtain pd.DataFrame
band = 'Gpp'
scale_factor = 0.0001
modis_gpp = extract_time_series(latitude,longitude,start_date,end_date,product,band,scale_factor)

<br>

#### Now we can create a figure to compare ELM GPP vs MODIS GPP retrievals for the study site

In [ ]:
timerange=slice('2010-01-01','2014-12-31')

fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1, 1, 1)
# need to convert ELM from gC/m^2/s to kgC/m2/8days
tempgppdata = (elm_output['GPP'])*86400
tempgppdata.sel(time=timerange).plot(ax=ax, linestyle='-',linewidth=2,color='black',label='ELM GPP')
temp_modis_gpp = np.divide(modis_gpp['Gpp']*1000, 8)
temp_modis_gpp.plot(ax=ax,linestyle='-',linewidth=2,color='green',label='MODIS GPP')
ax.legend(fontsize=15)
plt.xticks(fontsize=20, rotation=0, horizontalalignment='center')
plt.yticks(fontsize=20, rotation=0)
ax.set_title('GPP: 2010-2014', size=20)
ax.set_xlabel('Year', size=24)
ax.set_ylabel('C flux (gC m$^{-2}$ day$^{-1}$)', size=24)
if savefigs:
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_GPP.pdf'))
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_GPP.png'))
plt.show()

<br>

#### We can also extract a timeseries of Net Primary Productivity (NPP) data for the study site

In [ ]:
# Run the extract function
latitude = lat
longitude = lon
start_date = '2000-01-01'
end_date = '2014-12-31'
product = 'MODIS/006/MOD17A3HGF' 

# Extract data and obtain pd.DataFrame
band = 'Npp'
scale_factor = 0.0001
modis_npp = extract_time_series(latitude,longitude,start_date,end_date,product,band,scale_factor)

In [ ]:
print(modis_npp['Npp'])

<br>

#### Before we compare ELM to the MODIS NPP product, we will need to scale the units to match. As you can see below, NPP in ELM is currently a flux (gC/m2/2)

In [ ]:
elm_output['NPP']

<br>

#### Here we can select out an overlapping date range and then convert ELM from a flux to a pool

In [ ]:
# subset and convert from gC/m2/s to kgC/m2/day
dataset_sub = elm_output.sel(time=slice('1999', '2013'))
# calculate annual sum (i.e. annual total NPP)
elm_annual_npp = (dataset_sub['NPP']*0.001*86400).resample(time='Y').sum('time')

<br>

#### Now we can create the figure

In [ ]:
# Draw the comparison plot between MODIS and ELM annual NPP
fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1, 1, 1)
elm_annual_npp.plot(ax=ax, linestyle='-',linewidth=5,color='black',label='ELM NPP')
modis_npp['Npp'].plot(ax=ax,linestyle='-',linewidth=5,color='green',label='MODIS NPP')
ax.legend(fontsize=16)
plt.xticks(fontsize=20, rotation=0, horizontalalignment='center')
plt.yticks(fontsize=20, rotation=0)
ax.set_title('NPP: 2000-2014', size=20)
ax.set_xlabel('Date', size=24)
ax.set_ylabel('NPP (kg C m$^{-2}$)', size=24)
if savefigs:
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_Annual_NPP.pdf'))
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_Annual_NPP.png'))
plt.show()

<br>

#### Create a X-Y scatter plot of the annual NPP comparison between MODIS and ELM

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1, 1, 1)

plt.scatter(elm_annual_npp, modis_npp['Npp'], c="black", s=500)
ax.set_ylim([0.0, 0.55])
ax.set_xlim([0.0, 0.55])
plt.xticks(fontsize=20, rotation=0, horizontalalignment='center')
plt.yticks(fontsize=20, rotation=0)
ax.plot([0,1],[0,1], transform=ax.transAxes, linestyle='--', color='black')
ax.set_xlabel('ELM NPP (kg C m$^{-2}$)', size=20)
ax.set_ylabel('MODIS NPP (kg C m$^{-2}$)', size=20)
if savefigs:
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_Annual_NPP_scatter.pdf'))
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_Annual_NPP_scatter.png'))
plt.show()

<br>

#### We can also compare the annual NPP distributions in a histogram plot

In [ ]:
fig = plt.figure(figsize=(16, 8))
plt.hist(elm_annual_npp, alpha=0.5, label='ELM NPP', histtype='bar')
n, bins, patches = plt.hist(modis_npp['Npp'], alpha=0.5, label='MODIS NPP', histtype='bar')
plt.legend(fontsize=15)
plt.xlabel('NPP (kg C m$^{-2}$)', size=20)
plt.ylabel('Count', size=20)
plt.xticks(fontsize=16, rotation=0, horizontalalignment='center')
plt.yticks(fontsize=16, rotation=0)
if savefigs:
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_Annual_NPP_histogram.pdf'))
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_Annual_NPP_histogram.png'))
plt.show()

<br>

#### Derived variable example: Surface Albedo

Using ELM outputs, we can also derive new variables from standard model outputs.  We can calculate the surface albedo estiamted by ELM.  To do that we can calculate direct/diffuse, visible and near-infrared as well as full-sky (all-sky) albedo using the outputs availible in ELM.  In this case we will use the FSDS and FSR outputs to calculate surface reflectivity (i.e. albedo) from the ratio of reflected (FSR) to total incident radiation (FSDS)

In [ ]:
# Calculate direct VIS albedo
elm_output['BSA_VIS'] = elm_output['FSRVD']/elm_output['FSDSVD'].where(elm_output['FSDSVD']>0)
elm_output['BSA_VIS'].attrs['units'] = 'unitless'
elm_output['BSA_VIS'].attrs['long_name'] = 'Black Sky Visible Albedo'

# Calculate direct NIR albedo
elm_output['BSA_NIR'] = elm_output['FSRND']/elm_output['FSDSND'].where(elm_output['FSDSND']>0)
elm_output['BSA_NIR'].attrs['units'] = 'unitless'
elm_output['BSA_NIR'].attrs['long_name'] = 'Black Sky NIR Albedo'

# Calculate all-sky albedo and plot the timeseries
elm_output['ASA'] = elm_output['FSR']/elm_output['FSDS'].where(elm_output['FSDS']>0)
elm_output['ASA'].attrs['units'] = 'unitless'
elm_output['ASA'].attrs['long_name'] = 'All sky albedo'

<br>

#### Download MODIS albedo (direct, diffuse, VIS and NIR)

In [ ]:
# Run the extract function
latitude = lat
longitude = lon
start_date = '2012-01-01'
end_date = '2012-12-31'
product = 'MODIS/006/MCD43A3' 

# Extract data and obtain pd.DataFrame
band = 'Albedo_BSA_vis'
scale_factor = 0.001
modis_bsa_vis = extract_time_series(latitude,longitude,start_date,end_date,product,band,scale_factor)

band = 'Albedo_BSA_nir'
scale_factor = 0.001
modis_bsa_nir = extract_time_series(latitude,longitude,start_date,end_date,product,band,scale_factor)

band = 'Albedo_BSA_shortwave'
scale_factor = 0.001
modis_bsa_short = extract_time_series(latitude,longitude,start_date,end_date,product,band,scale_factor)

<br>

#### Create comparison plot of Black Sky Visible Albedo (BSA_VIS)

In [ ]:
timerange=slice('2012-01-01','2012-12-31')

fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1, 1, 1)
elm_output['BSA_VIS'].sel(time=timerange).plot(ax=ax, linestyle='-',linewidth=2.5,color='black',label='ELM VIS BSA')
modis_bsa_vis['Albedo_BSA_vis'].plot(ax=ax,linestyle='-',linewidth=2.5,color='green',label='MODIS VIS BSA')
ax.legend(fontsize=17)
plt.xticks(fontsize=20, rotation=0, horizontalalignment='center')
plt.yticks(fontsize=20, rotation=0)
ax.set_title('Vis Black Sky Albedo: 2012', size=18)
ax.set_xlabel('Year', size=24)
ax.set_ylabel('BSVIS Albedo (-)', size=24)
if savefigs:
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_BSA_VIS.pdf'))
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_BSA_VIS.png'))
plt.show()

<br>

#### Create comparison plot of Black Sky NIR Albedo (BSA_NIR)

In [ ]:
timerange=slice('2012-01-01','2012-12-31')

fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1, 1, 1)
elm_output['BSA_NIR'].sel(time=timerange).plot(ax=ax, linestyle='-',linewidth=2.5,color='black',label='ELM NIR BSA')
modis_bsa_nir['Albedo_BSA_nir'].plot(ax=ax,linestyle='-',linewidth=2.5,color='green',label='MODIS NIR BSA')
ax.legend(fontsize=17)
plt.xticks(fontsize=20, rotation=0, horizontalalignment='center')
plt.yticks(fontsize=20, rotation=0)
ax.set_title('NIR Black Sky Albedo: 2012', size=18)
ax.set_xlabel('Year', size=24)
ax.set_ylabel('BSNIR Albedo (-)', size=24)
if savefigs:
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_BSA_NIR.pdf'))
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_BSA_NIR.png'))
plt.show()

<br>

#### Create comparison plot of Black Sky Shortwave Albedo (BSA_Shortwave)

In [ ]:
timerange=slice('2012-01-01','2012-12-31')

fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1, 1, 1)
elm_output['ASA'].sel(time=timerange).plot(ax=ax, linestyle='-',linewidth=2.5,color='black',label='ELM Shortwave BSA')
modis_bsa_short['Albedo_BSA_shortwave'].plot(ax=ax,linestyle='-',linewidth=2.5,color='green',label='MODIS Shortwave BSA')
ax.legend(fontsize=17)
plt.xticks(fontsize=20, rotation=0, horizontalalignment='center')
plt.yticks(fontsize=20, rotation=0)
ax.set_title('Shortwave Black Sky Albedo: 2012', size=18)
ax.set_xlabel('Year', size=24)
ax.set_ylabel('BS Shortwave Albedo (-)', size=24)
if savefigs:
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_BSA_Shortwave.pdf'))
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_BSA_Shortwave.png'))
plt.show()

<br>

#### We can also evaluate the prediction of the water cycle against satellite retrievals.  

To do this analysis we can use the MODIS MOD16A2: MODIS Global Terrestrial Evapotranspiration 8-Day Global 1km product to evaluate ELM evapotranspiration and latent heat flux

In [ ]:
# Run the extract function
latitude = lat
longitude = lon
start_date = '2010-01-01'
end_date = '2014-12-31'
product = 'MODIS/NTSG/MOD16A2/105' 

# Extract data and obtain pd.DataFrame
band = 'ET'
scale_factor = 0.1
modis_et = extract_time_series(latitude,longitude,start_date,end_date,product,band,scale_factor)
#kg/m^2/8day

band = 'LE'
scale_factor = 10000
modis_latheat = extract_time_series(latitude,longitude,start_date,end_date,product,band,scale_factor)
#J/m^2/day

In [ ]:
# before we plot ET lets do some unit conversions
modis_et2 = np.divide(modis_et['ET'],8) # kg/m2/s

# we also need to calculate ELM ET in mm/s
elm_output['ET'] = elm_output['QVEGT']+elm_output['QVEGE']+elm_output['QSOIL']
elm_output['ET'].attrs['units'] = 'mm/s'
elm_output['ET'].attrs['long_name'] = 'Evapotranspiration'

<br>

#### Now that we have the MODIS estimates, lets create a plot comparing MODIS and ELM ET and LE.  

In [ ]:
#EFLX_LH_TOT
timerange=slice('2010-01-01','2014-12-31')

# Create the figure
fig,a=plt.subplots(nrows=2,ncols=1,clear=True,num='Radiation',figsize=(17,12))

ax=a[0]
elm_output['EFLX_LH_TOT'].sel(time=timerange).plot(ax=ax,linestyle='-',linewidth=2.5,color='green',label='ELM Latent Heat Flux')
# convert MODIS from J/m^2/day to W/m2 to match ELM
(modis_latheat['LE']*0.00001157407).plot(ax=ax,linestyle='-',linewidth=2.5,color='black',label='MODIS Latent Heat Flux')
ax.legend(fontsize=17)
ax.set_title('Latent Heat Flux: 2010-2014')
plt.xticks(fontsize=20, rotation=0, horizontalalignment='center')
plt.yticks(fontsize=20, rotation=0)
ax.set_xlabel('Date', size=24)
ax.set_ylabel('LE (W m$^{-2}$)', size=24)

ax=a[1]
(elm_output['ET']*24*3600).sel(time=timerange).plot(ax=ax,linestyle='-',linewidth=2.5,color='green',label='ELM Evapotranspiration')
modis_et2.plot(ax=ax,linestyle='-',linewidth=2.5,color='black',label='MODIS Evapotranspiration')
ax.legend(fontsize=17)
ax.set_title('Evapotranspiration: 2010-2014')
plt.xticks(fontsize=20, rotation=0, horizontalalignment='center')
plt.yticks(fontsize=20, rotation=0)
ax.set_xlabel('Date', size=24)
ax.set_ylabel('ET (mm s$^{-1}$ d$^{-1}$)', size=24)

if savefigs:
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_LE_ET.pdf'))
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_LE_ET.png'))
plt.show()

<br>

#### Compare snow pack dynamics with remotely-sensed estimates. Use both Terra and Aqua satellite retrievals

In [ ]:
# Run the extract function - get snow cover fraction from the Terra satellite
latitude = lat
longitude = lon
start_date = '2010-01-01'
end_date = '2012-12-31'
product = 'MODIS/006/MOD10A1' 

# Extract data and obtain pd.DataFrame
band = 'NDSI_Snow_Cover'
scale_factor = 1
modis_snow_cover = extract_time_series(latitude,longitude,start_date,end_date,product,band,scale_factor)

In [ ]:
# Run the extract function - get the same data from the Aqua satellite
latitude = lat
longitude = lon
start_date = '2010-01-01'
end_date = '2012-12-31'
product = 'MODIS/006/MYD10A1' 

# Extract data and obtain pd.DataFrame
band = 'NDSI_Snow_Cover'
scale_factor = 1
modisqa_snow_cover = extract_time_series(latitude,longitude,start_date,end_date,product,band,scale_factor)

<br>

#### Create the MODIS vs ELM ground snow cover fraction plot. Here we will use both the Terra and Aqua satellite data to enhance the comparison  

Note the issues with high-latitude satellite retrievals where a large proportion of the satellite timeseries is missing either because of cloud cover or other issues that impact the data quality

In [ ]:
timerange=slice('2009-10-01','2013-07-31')

fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1, 1, 1)
(elm_output['FSNO']*100).sel(time=timerange).plot(ax=ax, linestyle='-',linewidth=2.5,color='black',label='ELM Snow Fraction')
modis_snow_cover['NDSI_Snow_Cover'].plot(ax=ax, linestyle='-',linewidth=2.5,color='blue',label='MODIS Terra Snow Fraction')
modisqa_snow_cover['NDSI_Snow_Cover'].plot(ax=ax, linestyle='-',linewidth=2.5,color='green',label='MODIS Aqua Snow Fraction')
ax.legend(fontsize=15)
plt.xticks(fontsize=20, rotation=0, horizontalalignment='center')
plt.yticks(fontsize=20, rotation=0)
ax.set_title('Snow Fraction: 2010-2012', size=20)
ax.set_xlabel('Year', size=24)
ax.set_ylabel('Snow Fraction (%)', size=24)
if savefigs:
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_Snow_Fraction.pdf'))
    plt.savefig(os.path.join(figdir,'ELM_vs_MODIS_Snow_Fraction.png'))
plt.show()

<br>